<a href="https://colab.research.google.com/github/thor4/neuralnets/blob/master/projects/1-CNN/step_3_process_logits.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Process logits
---
Now we can load logits generated from each model and extract confidence and accuracy from them

## 1: Load logits
These logits were created using the `step2-generation_predictions-mobilenetv2_model.ipynb` and `step2-generation_predictions-vanilla_model.ipynb` Jupyter notebooks. Run the cell to download a zip file from OSF then extract the excel files `logits_mobilenetv2.xlsx` and `logits_vanilla.xlsx` into the current directory

In [1]:
# @title Download logits

import requests, os
from zipfile import ZipFile

print("Start downloading and unzipping `Model logits`...")
name = 'logits'
fname = f"{name}.zip"
url = f"https://osf.io/k8fwn/download" #osf share link
r = requests.get(url, allow_redirects=True)
with open(fname, 'wb') as fh:
  fh.write(r.content) #download file

with ZipFile(fname, 'r') as zfile:
  zfile.extractall() #extract contents

if os.path.exists(fname):
  os.remove(fname) #delete zip file
else:
  print(f"The file {fname} does not exist")

print("Download completed.")

Start downloading and unzipping `Model logits`...
Download completed.


Now we can load the logits into pandas dataframes

In [2]:
import pandas as pd

df_mnet = pd.read_excel("logits_mobilenetv2.xlsx") #mobilenetv2 logits
df_van = pd.read_excel("logits_vanilla.xlsx") #vanilla logits
print(df_mnet.head())
print(df_van.head())

     Logits  Labels
0  0.263464       0
1  0.380446       0
2 -0.105008       0
3  0.152609       0
4 -0.277837       0
     Logits  Labels
0 -2.320800       0
1 -0.269380       0
2 -2.161348       0
3 -0.795077       0
4 -2.513597       0


 -want to see how robust this is, take a range of confidence thresholds 
 -build out a function that takes n number of conditions/datasets and n number of logits and outputs the confidence and accuracy
After that, determine whether this model's logits produce the same dissociation results

-what about comparing the logits each model generates for each stimulus? some stimuli within datasets appear to drive confidence more than others. models do better with some stimulus images than others. maybe they specialize in a type of image

Now we will process every set of 1,000 logits as its own dataset from each dataframe
Let's explore the dataframe structure so we can break it down into its 18 constituent data sets

First, let's define the function to get confidence and accuracy

In [ ]:
thres_min = -0.5
thres_max = 0.5
#high_conf = df_mnet[(df_mnet['Logits'] < thres_min) | (df_mnet['Logits'] < thres_max)]
#_conf = df_mnet[(df_mnet['Logits'] < thres_min) | (df_mnet['Logits'] < thres_max)]
#print(high_conf.head())
#print(high_conf.shape)

     Logits  Labels
0  0.263464       0
1  0.380446       0
2 -0.105008       0
3  0.152609       0
4 -0.277837       0
(15531, 2)


Calculate confidence based on thresholds and add it to dataframe as a new column called `Confidence`

In [3]:
import numpy as np
thres_min = -0.5
thres_max = 0.5

conditions = [
    (df_mnet['Logits'] < thres_min) | (df_mnet['Logits'] > thres_max), #high conf
    (df_mnet['Logits'] >= thres_min) | (df_mnet['Logits'] <= thres_max) #low conf
]

values = ['high', 'low']

df_mnet['Confidence'] = np.select(conditions, values)

print(df_mnet.iloc[5000:5010])

        Logits  Labels Confidence
5000 -0.126332       0        low
5001 -1.941892       0       high
5002 -1.478281       0       high
5003 -1.361156       0       high
5004 -0.520181       0       high
5005  0.062038       0        low
5006 -0.320940       0        low
5007 -0.122390       0        low
5008 -1.939577       0       high
5009 -1.346852       0       high


In [8]:
df_mnet['Logits'].to_numpy()

array([ 0.26346406,  0.38044614, -0.10500798, ...,  5.1039381 ,
        5.38165665,  5.02641058])

In [14]:
from scipy.special import expit as sigmoid #get sigmoid function
df_mnet['Predictions'] = df_mnet['Logits'].apply(sigmoid) #sigmoid-transform the logits



In [15]:
df_mnet.iloc[5000:5010]

,Logits,Labels,Confidence,Predictions
5000,-0.126332,0,low,0.468459
5001,-1.941892,0,high,0.125440
5002,-1.478281,0,high,0.185687
5003,-1.361156,0,high,0.204053
5004,-0.520181,0,high,0.372810
5005,0.062038,0,low,0.515505
5006,-0.320940,0,low,0.420447
5007,-0.122390,0,low,0.469441
5008,-1.939577,0,high,0.125694
5009,-1.346852,0,high,0.206385


In [16]:
df_mnet['Predictions'] = np.where((df_mnet['Predictions'] < 0.5), 0, 1)
df_mnet.iloc[5000:5010]

,Logits,Labels,Confidence,Predictions
5000,-0.126332,0,low,0
5001,-1.941892,0,high,0
5002,-1.478281,0,high,0
5003,-1.361156,0,high,0
5004,-0.520181,0,high,0
5005,0.062038,0,low,1
5006,-0.320940,0,low,0
5007,-0.122390,0,low,0
5008,-1.939577,0,high,0
5009,-1.346852,0,high,0


In [17]:
df_mnet['Accuracy'] = np.where((df_mnet['Labels'] == df_mnet['Predictions']), 1, 0)
df_mnet.iloc[5000:5010]

,Logits,Labels,Confidence,Predictions,Accuracy
5000,-0.126332,0,low,0,1
5001,-1.941892,0,high,0,1
5002,-1.478281,0,high,0,1
5003,-1.361156,0,high,0,1
5004,-0.520181,0,high,0,1
5005,0.062038,0,low,1,0
5006,-0.320940,0,low,0,1
5007,-0.122390,0,low,0,1
5008,-1.939577,0,high,0,1
5009,-1.346852,0,high,0,1


In [ ]:
predictions = tf.nn.sigmoid(predictions) #apply sigmoid activation function to transform logits to [0,1]
predictions = tf.where(predictions < 0.5, 0, 1) #round down or up accordingly since it's a binary classifier
accuracy = tf.where(tf.equal(predictions,label_batch),1,0) #correct is 1 and incorrect is 0

In [ ]:
def age_groups(x):
    if x < 20:
        return '<20 years old'
    elif x < 40:
        return '20-39 years old'
    elif x < 60:
        return '40-59 years old'
    else:
        return '60+ years old'

df['Age Group'] = df['Age'].apply(age_groups)

print(df)

##NEXT: refactor and generalize code into get_conf_acc function

In [ ]:
def get_conf_acc(df, model, thres_min, thres_max):
    all_conf=tf.zeros([], tf.float64) #initialize array to hold all confidence ratings based on logit threshold (single element)
    all_acc=tf.zeros([], tf.float64) #initialize array to hold all accuracy indicators (single element)
    df[(df['Logits'] < thres_min) | (df['Logits'] < thres_max)]
    print(or_operator.head())
    print(or_operator.shape)
    predictions = model.predict_on_batch(image_batch).flatten() #run batch through model and return logits
    threshold = tf.math.logical_or(predictions < logit_thres_min, predictions > logit_thres_max) #set conf threshold at -20 and 20
    and_operator = df[(df['Sales'] > 300) & (df['Units'] > 20)]
    print(and_operator.head())
    print(and_operator.shape)
    or_operator = df[(df['Sales'] > 300) | (df['Units'] > 20)]
    print(or_operator.head())
    print(or_operator.shape)
    confidence = tf.where(threshold, 1, 0) #low confidence is 0, high confidence is 1 (based on logits)
    all_conf = tf.experimental.numpy.append(all_conf, confidence) #based on logit threshold
    all_pred = tf.experimental.numpy.append(all_pred, predictions)
    all_labels = tf.experimental.numpy.append(all_labels, label_batch)
    predictions = tf.nn.sigmoid(predictions) #apply sigmoid activation function to transform logits to [0,1]
    all_pred_sigtrans = tf.experimental.numpy.append(all_pred_sigtrans, predictions)
    threshold2 = tf.math.logical_or(predictions < 0.25, predictions > 0.75) #set conf threshold based on sigmoid-transformed logits, closer to 0 or 1 to track either class
    confidence2 = tf.where(threshold2, 1, 0) #low confidence is 0, high confidence is 1 based on sig-trans logits
    all_conf2 = tf.experimental.numpy.append(all_conf2, confidence2) #based on sig-trans threshold
    predictions = tf.where(predictions < 0.5, 0, 1) #round down or up accordingly since it's a binary classifier
    all_pred_final = tf.experimental.numpy.append(all_pred_final, predictions)
    accuracy = tf.where(tf.equal(predictions,label_batch),1,0) #correct is 1 and incorrect is 0
    all_acc = tf.experimental.numpy.append(all_acc, accuracy)
    if all_acc.size==BATCH_SIZE+1: #accumulating avg loop, check to see if on first loop
        avg_accuracy = tf.reduce_mean(tf.dtypes.cast(accuracy, tf.float64)) #avg performance, init avg acc of first batch
    else:
        #average last/running avg with current batch's avg:
        avg_accuracy = tf.reduce_mean(tf.concat([avg_accuracy,tf.reduce_mean(tf.dtypes.cast(accuracy, tf.float64))],0))
    #save current accuracy in array
    all_avg_acc = tf.experimental.numpy.append(all_avg_acc, tf.reduce_mean(tf.dtypes.cast(accuracy, tf.float64)))
    #tf.size(all_conf) #1335 elements, 1334 images + 1 placeholder 0 at beginning
    all_conf = all_conf[1:]
    all_conf2 = all_conf2[1:]
    all_pred = all_pred[1:]
    all_pred_sigtrans = all_pred_sigtrans[1:]
    all_pred_final = all_pred_final[1:]
    all_acc = all_acc[1:]  
    all_labels = all_labels[1:]
    all_avg_acc = all_avg_acc[1:] #drop first placeholder element
    high_conf_avg = tf.reduce_mean(all_conf) #avg of high conf
    high_conf_avg2 = tf.reduce_mean(all_conf2) #avg of high conf based on sig-trans logits
    avg_acc = tf.reduce_mean(all_acc) #avg performance
    batch_avg_acc = tf.reduce_mean(all_avg_acc) #avg avg performance on each batch
    loss, tf_accuracy = model.evaluate(test_dataset) #get TF model.evaluate() accuracy
    # print('Predictions:\n', all_pred_sigtrans[:].data.numpy())
    # print('Labels:\n', all_labels.data.numpy()) #nice job predicting
    # print('Incorrect:\n', all_acc.data.numpy())
    # print('High Confidence (Logits):', high_conf_avg.numpy()) #using logits threshold
    # print('High Confidence (Sig-trans):', high_conf_avg2.numpy()) #using sig-transformed threshold
    # print('Accuracy:', avg_acc.numpy()) #base: 0.965 vs 0.9528 with model.evaluate (maybe the sigmoid or rounding steps are diff)
    # print('Batch Accuracy:', batch_avg_acc.numpy()) #overall avg of each batches' avg performance, same as avg_acc
    # print('Accumulating Avg Accuracy:', avg_accuracy.numpy())
    # print('TF model.evaluate() Accuracy:', tf_accuracy)
    ## convert your array into a dataframe
    df = pd.DataFrame({"High Conf (logits)":all_conf.data.numpy(),"High Conf (sig-trans)":all_conf2.data.numpy(),
                   "Prediction (logits)":all_pred.data.numpy(),"Prediction (sig-trans)":all_pred_sigtrans.data.numpy(),
                   "Prediction (rounded)":all_pred_final.data.numpy(),"Actual Label":all_labels.data.numpy(),"Performance":all_acc.data.numpy()}) #this makes a dataframe of all stimuli responses
    df_results = pd.DataFrame({"High Conf (logits) Avg":high_conf_avg.numpy(),"High Conf (sig-trans) Avg":high_conf_avg2.numpy(),
                               "My Acc":avg_acc.numpy(),"Batch Acc":batch_avg_acc.numpy(),"Acc Avg Acc":avg_accuracy.numpy(),
                               "TF model.eval Acc":tf_accuracy}, index=[0])
    return df,df_results,all_avg_acc,avg_accuracy

In [ ]:
df_mnet.shape

(18000, 2)

In [ ]:
df_mnet.iloc[999]

Logits   -0.072107
Labels    1.000000
Name: 999, dtype: float64

In [ ]:
test = df_mnet[0:1000] #first 1,000 entries
df_mnet[0:1000].shape

(1000, 2)

In [ ]:
test.iloc[-1] #last entry

Logits   -0.072107
Labels    1.000000
Name: 999, dtype: float64

last entry in `test` is same as the 999th entry of `df_mnet` since index starts at 0

In [ ]:
test.iloc[-1] == df_mnet.iloc[999] 

Logits    True
Labels    True
Name: 999, dtype: bool

the 17,999 entry in `df_mnet` is the last since index starts at 0

In [ ]:
df_mnet.iloc[17999] == df_mnet.iloc[-1]

Logits    True
Labels    True
Name: 17999, dtype: bool

In [ ]:
test2 = df_mnet[1000:2000]
df_mnet[1000:2000].shape

(1000, 2)

In [ ]:
test2.iloc[0]

Logits   -0.590789
Labels    0.000000
Name: 1000, dtype: float64

Let's establish our index ranges.

In [ ]:
span = range(0, 18000, 1000)
for n in span:
  print(n,'to',n+1000)

0 to 1000
1000 to 2000
2000 to 3000
3000 to 4000
4000 to 5000
5000 to 6000
6000 to 7000
7000 to 8000
8000 to 9000
9000 to 10000
10000 to 11000
11000 to 12000
12000 to 13000
13000 to 14000
14000 to 15000
15000 to 16000
16000 to 17000
17000 to 18000


In [ ]:
span = range(0, 18000, 1000)
for idx in span:
  print(idx,'to',idx+1000)
  df_mnet[idx:idx+1000]